In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

24/04/16 15:48:12 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 10.90.9.242 instead (on interface en0)
24/04/16 15:48:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/16 15:48:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
directory = "/Users/mzc01-hyucksangcho/Downloads"
filename = "fhvhv_tripdata_2020-03.parquet"
data = spark.read.parquet(f"file:///{directory}/{filename}", inferSchema=True, header = True)

In [5]:
data.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [6]:
data.createOrReplaceTempView("mobility_data")

In [7]:
spark.sql("select * from mobility_data limit 5").show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [16]:
spark.sql("select split(pickup_datetime, ' ')[0] as pickup_date, count(*) as trips from mobility_data group by pickup_date").show(5)

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-16|391518|
| 2020-03-03|697880|
| 2020-03-06|872012|
| 2020-03-26|141607|
| 2020-03-05|731165|
+-----------+------+
only showing top 5 rows



In [17]:
zone_filename = "taxi_zone_lookup.csv"
zone_data = spark.read.csv(f"file:///{directory}/{zone_filename}", inferSchema=True, header = True)

In [20]:
data.createOrReplaceTempView("trip_data")
zone_data.createOrReplaceTempView("zone_data")

In [21]:
spark.sql("select * from trip_data limit 5").show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [22]:
spark.sql("select * from zone_data limit 5").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [25]:
spark.sql("select zone_data.Borough as borough, count(*) \
from trip_data join zone_data on trip_data.PULocationID = zone_data.LocationID \
group by zone_data.Borough").show()

+-------------+--------+
|      borough|count(1)|
+-------------+--------+
|       Queens| 2437394|
|          EWR|     362|
|     Brooklyn| 3735765|
|Staten Island|  178818|
|          N/A|     845|
|    Manhattan| 4953147|
|        Bronx| 2086597|
+-------------+--------+

